In [ ]:

import onnx
from easydict import EasyDict

from onnx2keras import onnx_to_keras
from onnx_pytorch import code_gen



import tensorflow_datasets as tfds

import numpy as np

import torchvision.models as models
import torch

import eagerpy as ep

from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD

import tensorflow as tf

# tf.compat.v1.disable_eager_execution()
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

from art.attacks.evasion import FastGradientMethod
from art.estimators.classification.keras import KerasClassifier
from art.estimators.classification import TensorFlowV2Classifier
from art.estimators.classification import PyTorchClassifier
from art.utils import load_mnist

from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method

In [ ]:
def sigmoid(input):
    return 1 / (1 + np.exp(-input))

In [ ]:
def onnx2Keras(path):
    onnx_model = onnx.load(path)
    model = onnx_to_keras(onnx_model, ['input_imgs', 'big_input_imgs', 'desire', 'traffic_convention', 'initial_state'],
                          name_policy='renumerate', change_ordering=False)
    model.save("models/keras" )

In [ ]:
onnx_model = "models/supercombo.onnx"
kerasModel = tf.keras.models.load_model("models/keras", compile=False)
pytorchModel = torch.load('models/supercombo.pt')
pytorchModel.eval()

In [ ]:
dataset = np.load('data/numpy.npz')

datasize = dataset['inputImgs'].shape[0]

inputImgs_data = dataset['inputImgs']
bigInputImgs_data = dataset['bigInputImgs']
desire_data = dataset['desire']
trafficConvention_data = dataset['trafficConvention']
initialState_data = dataset['initialState']
output_data = dataset['output']

Hieronder zullen alle tools getest worden.
Als eerste wordt het openpilot model getest en daarna wordt bij de laatste tools 2 ook van de bekende datasets getest om te laten zien wat tool doet doet.

Testing tool –Adversial Robustness 360 Toolbox–

In [ ]:
try:
    classifier = KerasClassifier(model=kerasModel, use_logits=False)
    attack = FastGradientMethod(estimator=classifier, eps=0.2)
    x_test_adv = attack.generate(x={"input_imgs": inputImgs_data,
                                    "big_input_imgs": bigInputImgs_data,
                                    "desire": desire_data,
                                    "traffic_convention": trafficConvention_data,
                                    "initial_state": initialState_data})
    predictions = classifier.predict(x_test_adv)
    accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
    print("Accuracy on adversarial test examples: {}%".format(accuracy * 100))
except Exception as e: print("Expected 1 input variable but got 5")

Testing tool –Cleverhans–

In [ ]:
try:
    test_acc_clean = tf.metrics.SparseCategoricalAccuracy()
    test_acc_fgsm = tf.metrics.SparseCategoricalAccuracy()
    for x, y in dataset:
        test_acc_clean(y, output_data)
        x_fgm = fast_gradient_method(kerasModel, x, 0.3, np.inf)
        y_pred_fgm = model(x_fgm)
        test_acc_fgsm(y, y_pred_fgm)

    print("test acc on clean examples (%): {:.3f}".format(test_acc_clean.result() * 100))
    print("test acc on FGM adversarial examples (%): {:.3f}".format(test_acc_fgsm.result() * 100))
except Exception as e: print("Expected 1 input variable but got 5")

Testing tool –Foolbox–

In [ ]:
try:
    model = "model/supercombo.pt"
    preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
    fmodel = PyTorchModel(model, bounds=(0, 1), preprocessing=preprocessing)

    images, labels = ep.astensors(*samples(fmodel, dataset="imagenet", batchsize=16))
    clean_acc = accuracy(fmodel, images, labels)
    print(f"clean accuracy:  {clean_acc * 100:.1f} %")

    attack = LinfPGD()
    epsilons = [
        0.0,
        0.0002,
        0.0005,
        0.0008,
        0.001,
        0.0015,
        0.002,
        0.003,
        0.01,
        0.1,
        0.3,
        0.5,
        1.0,
    ]
    raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilons)
    robust_accuracy = 1 - success.float32().mean(axis=-1)
    print("robust accuracy for perturbations with")
    for eps, acc in zip(epsilons, robust_accuracy):
        print(f"  Linf norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")
except Exception as e: print("Expected 1 input variable but got 5")